In [0]:
# Testing for GPU availability
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**1-Downloading Dataset**

In [0]:

# Installing database file/s
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls

In [0]:

# The classes Sequential and Dense are used to specify the nodes, connections, specifications of the neural network.
import keras
from keras.models import Sequential
from keras.layers import Dense

extracting data

In [0]:

import pickle
import gzip
import numpy

with open('2016.04C.multisnr.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()

In [0]:

#Kareem
#alternative method (like paper code)
import numpy as np
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], p.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(p[(mod,snr)])
        for i in range(p[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)
print(X.shape)
print("one sample :")
print(X[1])

making sense of structure of data:

In [0]:


#Kareem
#all keys
print("keys:")
print(p.keys())
#print(p["QPSK",2].shape)
print("shape of all data:")
print(X.shape)
#list of all shapes:
shapes=[]
snrs=[]
mod_types=[]
for x in p.keys():
  shapes.append(p[x].shape)
  snrs.append(x[1])
  mod_types.append(x[0])
print("same distribution of number of samples as shown:")
print(sorted(shapes)[0:20])
print(sorted(shapes)[20:40])
print(sorted(shapes)[40:60])

print("same modulation type but different SNRs:")
print(p["QPSK",-20].shape)
print(p["QPSK",-18].shape)
print(p["QPSK",-16].shape)
print(p["QPSK",-14].shape)
print(p["QPSK",-12].shape)
print(p["QPSK",-10].shape)
print(p["QPSK",-8].shape)
print(p["QPSK",-6].shape)
print(p["QPSK",-4].shape)
print(p["QPSK",-2].shape)
print(p["QPSK",0].shape)
print(p["QPSK",2].shape)
print(p["QPSK",4].shape)
print(p["QPSK",6].shape)
print(p["QPSK",8].shape)
print(p["QPSK",10].shape)
print(p["QPSK",12].shape)
print(p["QPSK",14].shape)
print(p["QPSK",16].shape)
print(p["QPSK",18].shape)



print("Again : same modulation type but different SNRs:")
print(p["CPFSK",-20].shape)
print(p["CPFSK",-18].shape)
print(p["CPFSK",-16].shape)
print(p["CPFSK",-14].shape)
print(p["CPFSK",-12].shape)
print(p["CPFSK",-10].shape)
print(p["CPFSK",-8].shape)
print(p["CPFSK",-6].shape)
print(p["CPFSK",-4].shape)
print(p["CPFSK",-2].shape)
print(p["CPFSK",0].shape)
print(p["CPFSK",2].shape)
print(p["CPFSK",4].shape)
print(p["CPFSK",6].shape)
print(p["CPFSK",8].shape)
print(p["CPFSK",10].shape)
print(p["CPFSK",12].shape)
print(p["CPFSK",14].shape)
print(p["CPFSK",16].shape)
print(p["CPFSK",18].shape)

print("same SNR for Different modulation types:")
print(p["QPSK",2].shape)
print(p["PAM4",2].shape)
print(p["AM-DSB",2].shape)
print(p["QAM64",2].shape)
print(p["AM-SSB",2].shape)
print(p["QAM16",2].shape)
print(p["8PSK",2].shape)
print("values in keys:")
print(sorted(set(snrs)))
print(len(set(snrs)))
print(set(mod_types))
print(len(set(mod_types)))
print("all different shapes:")
print(sorted(set(shapes)))

# Conclusion of dataset structure:

Dataset has 11 modulation types and 20 SNR values forming 220 different keys
each key has different number of samples

each Modulation type (ex : QPSK,8PSK ...etc) has same number of samples for different SNRs

the 128 dimentions represent values at each micro second of 128 micro seconds so these are samples in time

drivative is the slope of a straight line and measures change of y with change of x so I think it is just the difference between each two consecutive sample values (over change of x which is 1)

for real valued vectors the drivative is called the gradient (there is a function in numpy for that)

numpy's diff does the job also very well (as far as I know) we need only difference between dimentions and divided by change in time which is 1 so only difference is needed


In [0]:
#drivative
import numpy as np
X_npArray=np.array(X)
print("before differentiation (2nd sample):")
print(X_npArray[0])
print("After diffrentiation : (you can see 1st element of any channel is diffrence between 1st and second of array before)")
X_driv=np.diff(X_npArray)
print(X_driv[0])
z=np.zeros((162060,2,1))
print("shape before:")
print(X_driv.shape)
X_driv=np.concatenate((z, X_driv), axis=2)
print("shape after:")
print(X_driv.shape)
print("element with added zeros :")
print(X_driv[0])
print(X[0])

## Integration using trapzoidal cummulative integration
### setting initial value to zero to make arrays match in size ((127+1,128))

In [0]:
import numpy as np
from scipy import integrate

X_npArray=np.array(X)
#############################################
S=integrate.cumtrapz(X_npArray, initial=0)

In [0]:
#add dimention of zeros

X_play=X_driv
y=np.zeros((162060,2,1))
z=np.concatenate((y, X_play), axis=2)
print(z.shape)
print(z)

In [0]:
combined_Data=np.zeros((162060,6,128))
print(combined_Data.shape)
for (i) in range(len(combined_Data)):
    combined_Data[i]=np.vstack((X[i],X_driv[i],S[i]))
print(combined_Data[0])

2 dimentions vectors because complex values a vector for real value and imaginary value

In [0]:
# Making the Confusion Matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(..., ...) 

**The actual Network**

In [0]:
!pip install keras 
!pip install tflearn
!pip install tensor

loading samples from pickle file

In [0]:
%matplotlib inline
import os,random
#os.environ["KERAS_BACKEND"] = "theano"
os.environ["KERAS_BACKEND"] = "tensorflow"
#os.environ["THEANO_FLAGS"]  = "device=cuda,floatX=float32"
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
import matplotlib.pyplot as plt
import seaborn as sns
import pickle, random, sys, keras
# Installing database file/s
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls
# The classes Sequential and Dense are used to specify the nodes, connections, specifications of the neural network.

with open('2016.04C.multisnr.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    Xd = u.load()
    


In [0]:
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

diffrentiation

In [0]:
#Kareem trying accuracy for differentiated data, uncomment the cell to see results
'''
import numpy as np
X_npArray=np.array(X)

X=np.diff(X_npArray)
'''

dividing data into training and testing sets

In [0]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0,n_examples), size= int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
#changed from x to combined data
#X_train = combined_Data[train_idx]
#X_test =  combined_Data[test_idx]
X_train = X[train_idx]
X_test =  X[test_idx]

one hot encoding for multiclass classification since there are 11 classes (11 modulation techniques)

In [0]:
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1 
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [0]:
in_shp = list(X_train.shape[1:])
print (X_train.shape, in_shp)
classes = mods

parameters and structure of network

In [0]:
# Set up some params 
nb_epoch = 100    # number of epochs to train on
batch_size = 700  # training batch size

In [0]:
#fully connected neural network
dr = 0.1
model1 = keras.models.Sequential()
model1.add(Reshape(in_shp+[1], input_shape=in_shp))
model1.add(Dropout(dr))
model1.add(Flatten())
model1.add(Dense(128, activation='relu', kernel_initializer='he_normal', name="dense1"))
model1.add(Dropout(dr))
model1.add(Dense(128, activation='relu', kernel_initializer='he_normal', name="dense2"))
model1.add(Dropout(dr))
model1.add(Dense(64, activation='relu', kernel_initializer='he_normal', name="dense3"))
model1.add(Dropout(dr))
model1.add(Dense(64, activation='relu', kernel_initializer='he_normal', name="dense4"))
model1.add(Dropout(dr))
model1.add(Dense( len(classes), kernel_initializer='he_normal', name="dense5" ))
model1.add(Activation('softmax'))
model1.add(Reshape([len(classes)]))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

In [0]:
# perform training ...
#   - call the main training loop in keras for our network+dataset
filepath = 'convmodrecnets_CNN2_0.5.wts.h5'
history=model1.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
     validation_split=0.05,
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
    ])
# we re-load the best weights once training is finished
model1.load_weights(filepath)

In [0]:
score = model1.evaluate(X_test, Y_test, batch_size=batch_size)
print(model1.metrics_names)
print (score)

In [0]:
import keras.models as models
# Build VT-CNN2 Neural Net model using Keras primitives -- 
#  - Reshape [N,2,128] to [N,1,2,128] on input
#  - Pass through 2 2DConv/ReLu layers
#  - Pass through 2 Dense layers (ReLu and Softmax)
#  - Perform categorical cross entropy optimization
 
dr = 0.1 # dropout rate (%)
model = keras.models.Sequential()
model.add(Reshape(in_shp+[1], input_shape=in_shp))
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(64, (1, 3),padding='valid', activation="relu", name="conv1",kernel_initializer='glorot_uniform',data_format="channels_last"))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(16,(2, 3), padding='valid', activation="relu", name="conv2", kernel_initializer='glorot_uniform',data_format="channels_last"))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
model.add(Dropout(dr))
model.add(Dense( len(classes), kernel_initializer='he_normal', name="dense2" ))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

training the model :

In [0]:
# perform training ...
#   - call the main training loop in keras for our network+dataset
filepath = 'convmodrecnets_CNN2_0.5.wts.h5'
model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
     validation_split=0.05,
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
    ])
# we re-load the best weights once training is finished
model.load_weights(filepath)

testing the model:

In [0]:
scscore = model.evaluate(X_test, Y_test, batch_size=batch_size)
print(model.metrics_names)
print (scscore)

visuals :

In [0]:
# Show loss curves 
plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()

In [0]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [0]:
# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
plot_confusion_matrix(confnorm, labels=classes)
# on the right its a color chart to indicate that the darker the color the easier and thee most to identify

In [0]:
# Plot confusion matrix
acc = {}
for snr in snrs:

    # extract classes @ SNR
    test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
    test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]    

    # estimate classes
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(classes)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    plt.figure()
    plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print ("Overall Accuracy: ", cor / (cor+ncor))
    acc[snr] = 1.0*cor/(cor+ncor)

In [0]:
# Plot accuracy curve
plt.plot(snrs, list(map(lambda x: acc[x], snrs)))
plt.xlabel("Signal to Noise Ratio")
plt.ylabel("Classification Accuracy")
plt.title("CNN2 Classification Accuracy on RadioML 2016.10 Alpha")